In [1]:

using LinearAlgebra
using GenericLinearAlgebra


In [2]:
function complete_orthonormal(U::Matrix{Float64})
    d, r = size(U)
    r ≤ d || error("U deve ter no máximo d colunas")

    r == d && return zeros(Float64, d, 0)   # já completa

    # Projeta vetores aleatórios no complemento de span(U)
    Z = randn(Float64, d, d - r)
    Z .-= U * (U' * Z)                      # remove componente em U

    # QR fino garante ortonormalidade
    C = Matrix(qr(Z).Q)                  # d × (d−r)
    return C
end

complete_orthonormal (generic function with 1 method)

In [3]:
function build_csd_input_matrix(D1::Matrix{Float64}, D2::Matrix{Float64})
    # Dimensões
    m, d = size(D1)  # D1: m x d
    n, _ = size(D2)  # D2: n x d

    # Passo 1: QR de D1^T para obter base de R(D1) em R^d
    Q1 = Matrix(qr(D1').Q)  # Q1 é d x m
    R1 = Matrix(qr(D1').R)  # R1 é m x m

    println("Q1: ", size(Q1))  # Debugging: verificar tamanho de Q1
    println("R1: ", size(R1))  # Debugging: verificar tamanho de R1

    r1 = sum(abs.(diag(R1)) .> eps() * maximum(abs.(diag(R1))))  # Estimar posto
    E1 = Q1[:, 1:r1]  
    E2 = complete_orthonormal(E1)  # Base ortonormal completa para R^d, d x (d - r1)
    E = hcat(E1, E2)  # Base ortonormal completa para R^d, d x d
    
    # Passo 2: QR de D2^T para obter base de R(D2) em R^d
    Q2 = Matrix(qr(D2').Q)  # Q2 é d x n
    R2 = Matrix(qr(D2').R)  # R2 é n x n

    println("Q2: ", size(Q2))  # Debugging: verificar tamanho de Q2
    println("R2: ", size(R2))  # Debugging: verificar tamanho de R2

    c1 = sum(abs.(diag(R2)) .> eps() * maximum(abs.(diag(R2))))  # Estimar posto
    F1 = Q2[:, 1:c1]  
    F2 = complete_orthonormal(F1)  # Base ortonormal completa para R^d, d x (d - c1)
    

    # Passo 3: Calcular Q11 = E1^T F1
    Q11 = E1' * F1  # r1 x c1
    Q12 = E1' * F2  # r1 x (d - c1)
    Q21 = E2' * F1  # (d - r1) x c1
    Q22 = E2' * F2  # (d - r1) x (d - c1)

    # Passo 4: Concatenar Q11, Q12, Q21, Q22
    Qtop = hcat(Q11, Q12)
    Qbot = hcat(Q21, Q22)
    Q = vcat(Qtop, Qbot)
    
    return Q, r1, c1, E, E1, F1, R1, R2
end

build_csd_input_matrix (generic function with 1 method)

In [4]:
# Exemplo com dados aleatórios
n = 100  # amostras dos datasets
d = 500  # número de features

D1 = randn(n, d)  # dataset 1
D2 = randn(n, d)  # dataset 2

Q, r1, c1, E, E1, F1, R1, R2 = build_csd_input_matrix(D1, D2)

Q1: (500, 100)
R1: (100, 100)
Q2: (500, 100)
R2: (100, 100)


([-0.014003922476514517 -0.05945429347303622 … -0.027439112457439227 -0.08627967111580956; -0.011423566931296682 0.007323678983271846 … -0.04890430067753271 0.018624065374738808; … ; 0.014762562284929148 0.04071855730227345 … 0.0013079928925760279 -0.05938360358870183; 0.02525442065569636 -0.010851209070244613 … 0.0161647956594158 -0.012255272328276315], 100, 100, [-0.015263470770990217 -0.04583765891621392 … 0.006756460667145292 -0.0302877228570489; -0.003998185609731355 -0.015226872396120639 … 0.0376851309984477 0.011017793661546968; … ; 0.06038237759429183 0.05719797414091383 … -0.005966225228594461 -0.023616928188778058; 0.09608370335432477 0.06703923083072505 … -0.05063462622996573 0.04098320893431617], [-0.015263470770990217 -0.04583765891621392 … 0.02409922602560376 -0.023723806021884928; -0.003998185609731355 -0.015226872396120639 … 0.0013662556103126485 0.07943600921095266; … ; 0.06038237759429183 0.05719797414091383 … 0.024861313944572846 0.02218422472221484; 0.09608370335432

In [5]:
display(size(Q))  # Verifica o tamanho da matriz Q
println("Posto de D1: ", r1)
println("Posto de D2: ", c1)
println("Tamanho de E: ", size(E))  # Verifica o tamanho da base ortonormal completa E
println("Tamanho de E1: ", size(E1))  # Verifica o tamanho de E1
println("Tamanho de F1: ", size(F1))  # Verifica o tamanho de E2
println("Tamanho de R1: ", size(R1))  # Verifica o tamanho de R1
println("Tamanho de R2: ", size(R2))  # Verifica o tamanho de R2

(500, 500)

Posto de D1: 100
Posto de D2: 100
Tamanho de E: (500, 500)
Tamanho de E1: (500, 100)
Tamanho de F1: (500, 100)
Tamanho de R1: (100, 100)
Tamanho de R2: (100, 100)


In [6]:
# Verificar se R1' * E1' é aproximadamente igual a D1
if R1'*E1' ≈ D1
    println("R1' * E1' é aproximadamente igual a D1")
else
    println("R1' * E1' é diferente de D1")
end

R1' * E1' é aproximadamente igual a D1


In [7]:
function rebuild_datasets_matrices(E, R)
    D_reconstructed = R' * E'  
    # Retornar a matriz reconstruída
    return D_reconstructed
end

rebuild_datasets_matrices (generic function with 1 method)

In [8]:
function verify_Q(Q::Matrix{Float64})
    if Q'*Q ≈ I
        println("A matriz Q é ortonormal")
    else
        println("A matriz Q não é ortonormal")
        
    end
end

verify_Q (generic function with 1 method)

In [9]:
# Exibir e verificar a corretude da matriz A
verify_Q(Q)
D1_reconstructed = rebuild_datasets_matrices(E1, R1)
D2_reconstructed = rebuild_datasets_matrices(F1, R2)

if D1_reconstructed ≈ D1 && D2_reconstructed ≈ D2
    println("As matrizes reconstruídas são equivalentes às originais!")
else
    println("As matrizes reconstruídas NÃO são equivalentes às originais.")
        
end

A matriz Q é ortonormal
As matrizes reconstruídas são equivalentes às originais!


In [10]:
function csd(Q, c1, r1, E)
    d= size(Q, 1)  # Dimensão do espaço
    # separar Q em blocos
    Q11 = Q[1:r1, 1:c1]  # r1 x c1
    Q12 = Q[1:r1, c1+1:end]  # r1 x (d - c1)
    Q21 = Q[r1+1:end, 1:c1]  # ( d - r1) x c1
    Q22 = Q[r1+1:end, c1+1:end]  # (d - r1) x (d - c1)

    # SVD dos blocos de Q = EᵀF
    U1, S11, V1 = svd(Q11; full=true)     
    U2, S22, V2  = svd(Q22; full=true)

    D11 = U1' * Q11 * V1
    D12 = U1' * Q12 * V2 
    D21 = U2' * Q21 * V1
    D22 = U2' * Q22 * V2
    
    # Monta matriz diagonalizada D via blocos projetados
    D_top = hcat(D11, D12)  # r1 x d
    D_bot = hcat(D21, D22)  # (d - r1) x (d - c1)
    D = vcat(D_top, D_bot)  # d × d

    # Constrói matriz U unificada
    U = Matrix{Float64}(I, d, d)
    U[1:r1, 1:r1] = U1
    U[r1+1:end, r1+1:end] = U2
    
    # Constrói matriz V unificada
    V= Matrix{Float64}(I, d, d)
    V[1:c1, 1:c1] = V1
    V[c1+1:end, c1+1:end] = V2
    
    # Calcula os ângulos θ entre os subespaços
    θ = acos.(clamp.(S11, -1.0, 1.0))

    # Reconstrói a rotação W que leva E1 → F1
    W = E * U * D * U' * E'


    return U, U1, U2, V, V1, V2, D, θ, W  
end

csd (generic function with 1 method)

In [11]:
U, U1, U2, V, V1, V2, D, θ, W =csd(Q, c1, r1, E)

([-0.021631568347529787 -0.03942064000241097 … 0.0 0.0; -0.08450141205358448 -0.08659102271317887 … 0.0 0.0; … ; 0.0 0.0 … 0.03143130660329895 -0.03565688723509688; 0.0 0.0 … 0.11763781926782998 -0.01860623876925731], [-0.021631568347529787 -0.03942064000241097 … -0.09770401664021885 -0.013979710778544478; -0.08450141205358448 -0.08659102271317887 … -0.1299765967301634 0.20238769333614542; … ; 0.11030243359740227 -0.15336227551569204 … -0.10007180545292367 0.11520921054087918; 0.19655085080303844 0.09801472334464091 … -0.07903104322026348 -0.026856617323851306], [-0.00621876641854346 0.02982165573329139 … 0.12185517160755235 -0.034169230037943; -0.010839156222335432 -0.014947125725252619 … -0.07533300453547172 -0.07340452144189041; … ; -0.05338371257743557 0.08374468304492586 … 0.03143130660329895 -0.03565688723509688; 0.00014044400903125143 -0.09510989187030333 … 0.11763781926782998 -0.01860623876925731], [-0.05272760046406454 0.17502425017928683 … 0.0 0.0; 0.16945482627758088 -0.1125

In [12]:
θ_deg = θ .* (180 / π)
println("Ângulos principais entre subespaços (em graus): ", θ_deg)
println("tamanho de U: ", size(U))
println("tamanho de U1: ", size(U1))
println("tamanho de U2: ", size(U2))
println("tamanho de V: ", size(V))
println("tamanho de V1: ", size(V1))
println("tamanho de V2: ", size(V2))
println("tamanho de D: ", size(D))
println("tamanho de W: ", size(W))

Ângulos principais entre subespaços (em graus): [35.35867299298733, 39.55807736594058, 40.17113712072999, 40.63744966266552, 41.3286696044899, 41.91464865998075, 43.03183540557737, 43.8496232491971, 44.588090056278816, 45.19720497362663, 46.021150801866874, 46.35679411685944, 47.239652699968595, 47.836101157005025, 48.192193675152645, 48.57048816575177, 48.97573946434506, 49.62034975910142, 50.16365440562102, 51.05590345515, 51.36735988963826, 52.225926582450306, 52.66804385084934, 53.14050573421529, 53.552503563874204, 54.305847755551, 54.82708109194573, 55.527260858615264, 56.29928395955354, 56.905974610453804, 57.409302446152196, 58.00919554803973, 58.67955687851828, 58.94147762525168, 59.15831069429074, 59.80424773158658, 60.00881406297375, 60.63614916313707, 61.1047532598787, 61.57946412587277, 62.23969549455474, 62.4291339855715, 62.74795977039058, 63.12402813416996, 63.69410761182387, 64.11482798987664, 64.27752381552378, 65.53002149486507, 65.63980181275419, 66.43579662600081, 

In [13]:
display(θ)                 # Ângulos principais entre subespaços (em radianos)
@show(U*D*V' ≈ Q) 
display(norm(W-I) ≈ norm(D-I))  # Deve ser próximo de 0 (rotação direta correta)
@show W*E1*U1*V1' ≈ F1 

100-element Vector{Float64}:
 0.6171252628636268
 0.6904186957943089
 0.7011186070268531
 0.7092572962269723
 0.7213213600672514
 0.7315486239332932
 0.7510472110035945
 0.7653203014575434
 0.778208978657809
 0.7888400394885416
 ⋮
 1.5040602609455145
 1.5100198957843096
 1.521673000968256
 1.5373079007699695
 1.545960209050115
 1.5486357476320243
 1.555546060958656
 1.558788208155831
 1.5684178273335758

true

U * D * V' ≈ Q = true
W * E1 * U1 * V1' ≈ F1 = true


true

In [14]:
D2_reconstructed_from_rotation = rebuild_datasets_matrices(W*E1*U1*V1', R2)

if  D2_reconstructed_from_rotation ≈ D2
    println("A matriz D2 reconstruída a partir da rotação é equivalente à original!")
else
    println("A matriz D2 reconstruída a partir da rotação NÃO é equivalente à original.")
end

A matriz D2 reconstruída a partir da rotação é equivalente à original!
